In [7]:
##Arcgis License
import arcpy
from arcpy.sa import *

ImportError: No module named sa

In [1]:
## set workspace (need to change)
arcpy.env.workspace = 'data/'
arcpy.env.overwriteOutput = True
##load extensions
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("Spatial")

NameError: name 'arcpy' is not defined

In [4]:
#Create LAS Dataset (3D Analyst or Spatial Analyst)
# input can be a folder if choose recursion
arcpy.CreateLasDataset_management('data/1662_2638.las', 'GH.lasd', 'RECURSION')

<Result 'data\\GH.lasd'>

In [5]:
#Classify LAS Building (3D Analyst) minHeight and minArea can change, but it generates automatically in Arcmap
arcpy.ClassifyLasBuilding_3d('GH.lasd', min_height='6.56 Feet', min_area='47.6 SquareFeet', compute_stats=True)

<Result 'data\\GH.lasd'>

In [6]:
#LAS Dataset to Raster (Spatial Analyst or 3D Analyst)
#FLOAT can change to INT if it takes long to run
arcpy.LasDatasetToRaster_conversion('GH.lasd', 'GH', "ELEVATION", "BINNING AVERAGE NATURAL_NEIGHBOR",
                                   "FLOAT", "CELLSIZE", 1, 1)

<Result 'data\\GH'>

In [7]:
#Aspect (3D Analyst or Spatial Analyst)
arcpy.Aspect_3d('GH', 'aspect')
#filter south facing or horizontal aspect. Flat, 112.5 <= aspect <= 247.5, set value to 1, others to None
#outSetNull = SetNull('aspect', 1, "VALUE < 0")
#outSetNull.save("filtered_aspect")
filter_aspect = Con((Raster('aspect') == -1) | (Raster('aspect') >= 112.5) & (Raster('aspect') <= 247.5), 1, '')
# save aspect_raster
filter_aspect.save("filtered_aspect.tif")

In [8]:
#Slope (3D Analyst or Spatial Analyst)
arcpy.Slope_3d('GH', 'slope', "DEGREE", 1)
# filter slope degree <= 35 to 1, others to None
filter_slope = Con(Raster('slope') <= 35, 1, '')
#save slope_raster
filter_slope.save("filtered_slope.tif")

In [ ]:
#Combine slope and aspect
Times("filtered_aspect.tif", "filtered_slope.tif")

In [ ]:
#Area Solar Radiation (Spatial Analyst)
# unit: watt hours per square meter (WH/m2) ??
solar = AreaSolarRadiation('GH', '', '', TimeWholeYear(2018))

In [ ]:
#Extract building polygon
arcpy.RasterToPolygon_conversion('GH', 'building', "NO_SIMPLIFY", "VALUE")

In [ ]:
#Hillshade (3D Analyst or Spatial Analyst)
#Haven't used yet
arcpy.HillShade_3d(raster, hillshaded_raster, 315, 45, "SHADOWS", 1)